# Datapreprocessing
* From CSV to Feather
* Drop NA values

In [1]:
#usual lines
import numpy as np
import pandas as pd

# import path
input_path = 'Data/us-equities_logreturns.csv'
output_path = 'Data/us-equities_logreturns.feather'

#load data
df = pd.read_csv(input_path)

#delete columns with NaN values
df_clean = df.dropna(axis=1, how="any", thresh=None, subset=None, inplace=False)
df_clean.drop(columns="Unnamed: 0",inplace=True)

# keep track of how many columns we have lost
columns_losses = df_clean.shape[1] - df.shape[1]

#write to feather 
df_clean.to_feather(output_path)

C:\Users\pahu95377\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# Louvain clustering for correlation matrices


In [2]:
from numpy import linalg as LA
import numpy as np
import pandas as pd
import math
import networkx as nx
import community


def compute_C_minus_C0(lambdas,v,lambda_plus):
    N=len(lambdas)
    C_clean=np.zeros((N, N))
  
    # _s stands for _structure below. Note that range(N-1) means that we do not include the maximum eigenvalue
    for i in range(N-1):
        if lambdas[i]>lambda_plus:
            C_clean=C_clean + lambdas[i]*(np.outer(v[:,i],v[:,i]))
    np.fill_diagonal(C_clean,1)
    return C_clean    
    
    
def LouvainCorrelationClustering(R):   # R is a matrix of return
    N=R.shape[1]
    T=R.shape[0]

    q=N*1./T
    lambda_plus=(1.+np.sqrt(q))**2

    C=R.corr()
    lambdas, v = LA.eigh(C)
    
    C_s=compute_C_minus_C0(lambdas,v,lambda_plus)
    C_s=np.abs(C_s)
    
    mygraph= nx.from_numpy_matrix(C_s)
    partition = community.community_louvain.best_partition(mygraph)

    DF=pd.DataFrame.from_dict(partition,orient="index")
    return(DF)

In [3]:
# Define clustering Assets
R=pd.read_feather("Data/us-equities_logreturns.feather")     
myclusteringAssets=LouvainCorrelationClustering(R)
myclusteringAssets.index=R.columns
print("Assets:\n",myclusteringAssets)
print("\nAssets values are:",np.unique(myclusteringAssets.values))

Assets:
      0
AEP  0
FL   1
MRO  2
DTE  0
IP   3
..  ..
SWM  8
ITT  3
MSM  0
HIG  2
BCO  4

[481 rows x 1 columns]

Assets values are: [0 1 2 3 4 5 6 7 8]


# Define a local period and compare the assets 
* R.shape  = (4549, 481) 
* each row = one day
* after a week t0 = 0 and t1= 7, we have decend numbers.
* do we have to keep tracks like 0

In [6]:
# set the time period
t0=1000 #starting period
t1=1051 #50 days on memory
Rlocal=R[t0:t1].copy()
myclusteringDays=LouvainCorrelationClustering(Rlocal)
myclusteringDays.index=Rlocal.columns
myclusteringDays.columns=["state"]
string_states = ''.join(map(str, myclusteringDays['state']))
print("List of states:\n",myclusteringDays)
print("\nUnique state values are:",np.unique(myclusteringDays.values))
print("\nThe string of states from t0:",t0,"and t1:",t1,"is:\n",string_states)

List of states:
      state
AEP      0
FL       1
MRO      2
DTE      0
IP       3
..     ...
SWM      1
ITT      3
MSM      5
HIG      1
BCO      1

[481 rows x 1 columns]

Unique state values are: [0 1 2 3 4 5 6 7]

The string of states from t0: 0 and t1: 1051 is:
 0120304351632205123002633331333205353311632332233651251223421132006137003011237151301032234253022333033303323173747644017003727111272261452424416416152640321463312032202300336313642311717515105660512715027313726127775300637042122616134424374113134413623344424611770324043464115174760656554271313310131111142123247501204512124573424235204421733734561422344447260024545111730574741074730556433664651115457276313215556574615555312353666354053556361516656131636266375722650210632713511
